<h3 style="margin-left:20px">Extended model v1 with  additional technical features- We choose the best features that gives us the lowest mean square error in the training set.
 
</h3>
 <ul class="list-group">
  <li class="list-group-item">Exponential Moving Average</li>
  <li class="list-group-item">Draw Down</li>
  <li class="list-group-item">Run Up</li>
  <li class="list-group-item">Volume Delta</li>
  <li class="list-group-item">Random Feature</li>
  <li class="list-group-item">Bollinger Bands</li>
</ul>

<h4 style="margin-left:20px">If anyone of the above features gave a lower mean square accruacy 
    in our base set, it was added to the set of initial features otherwise the features were 
    discarded.
    
</h4>

In [1]:
import random
import csv
try:
    import urllib.request as urllib2
except ImportError:
    import urllib2
from deap import base
from deap import creator
from deap import tools
from deap import algorithms
import pandas as pd
import numpy as np
#import QSTK.qstkfeat.featutil as ftu
from pyalgotrade import strategy
from pyalgotrade import barfeed
from pyalgotrade import bar
from pyalgotrade.technical import ma as pytechma
from pyalgotrade import plotter
from pyalgotrade.stratanalyzer import returns
from pyalgotrade.stratanalyzer import sharpe
from pyalgotrade.utils import stats as pytechstats
import datetime as dt
import os

In [2]:
# Change this to your directory structure

download_dir = '/Users/shadmanspc/Desktop/ADA/A2'

#'Specify the stock name in an array 

#Can do for any number of stocks just append the stock name in the stock array and download it
#it from a yahoo finance accordingly
#I took training from 2014-1-1 to 2018-12-31.

stocks = ['ASX.AX']

# keys from yahoo are different from what qstk library expects
#
yahoo_keys = ['Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']
qstk_keys = ['open', 'high', 'low', 'close', 'volume', 'actual_close']
#Keep track of the stocks and its data
raw_training_data = {}
# cloned for iteration, will remove those stocks from master stocks list if

# Initially designed for fetching direcrly from url but apparently everything has been deprecated.

stocks_clone = [s for s in stocks]

for s in stocks_clone:
    #Change the filename to stockname_training.csv
  
    file_name = download_dir + "/"+ s + '_training'+'.csv'
    print(file_name)
    if(os.path.isfile(file_name)) :
        raw_training_data[s] = pd.read_csv(file_name)
        
#I try to mimick the qstk library but this no longer available so I manipulate the data to my model
#to the way I believe qstk library would work
        

def sanitizedataforqstk(frame):
    data = {}
    for k in yahoo_keys:
        key_df = pd.DataFrame()
        for s in stocks:
            dates = np.array([dt.datetime.strptime(t, "%Y-%m-%d") for t in frame[s]['Date']])
            key_df[s] = pd.Series(data=frame[s][k].values, index=dates).sort_index(ascending=True)
        data[k] = key_df

    # sanitize to use the earlier keys
    data[qstk_keys[0]] = data[yahoo_keys[0]]
    data[qstk_keys[1]] = data[yahoo_keys[1]]
    data[qstk_keys[2]] = data[yahoo_keys[2]]
    data[qstk_keys[3]] = data[yahoo_keys[3]]
    data[qstk_keys[4]] = data[yahoo_keys[4]]
    data[qstk_keys[5]] = data[yahoo_keys[5]]
    del data[yahoo_keys[0]]
    del data[yahoo_keys[1]]
    del data[yahoo_keys[2]]
    del data[yahoo_keys[3]]
    del data[yahoo_keys[4]]
    del data[yahoo_keys[5]]
    for key in qstk_keys:
        data[key] = data[key].fillna(method='ffill')
        data[key] = data[key].fillna(method='bfill')
        data[key] = data[key].fillna(1.0)

    return data

training_data = sanitizedataforqstk(raw_training_data)
#print(training_data)

/Users/shadmanspc/Desktop/ADA/A2/ASX.AX_training.csv


In [3]:
#This code is copied from https://github.com/tuckerbalch/QSTK/blob/master/qstkfeat/features.py
#This library does is not supported anymore so I do my manipulation
#This code calculates moving average
def featMA( dData, lLookback=30, bRel=True, b_human=False ):
    '''
    @summary: Calculate moving average
    @param dData: Dictionary of data to use
    @param lLookback: Number of days to look in the past
    @param b_human: if true return dataframe to plot
    @return: DataFrame array containing values
    '''
    
    dfPrice = dData['close']
    
    dfRet = dfPrice.rolling(lLookback).mean()
    #ts_log.rolling(12).mean()
    
    if bRel:
        dfRet = dfRet / dfPrice
    if b_human:  
        data2 = dfRet * dData['close']
        data3 = pand.DataFrame({"Raw":data2[data2.columns[0]]})
        for sym in dfRet.columns:
            if sym != '$SPX' and sym != '$VIX':
                data3[sym + " Moving Average"] = data2[sym]
                data3[sym] = dData['close'][sym]
        del data3['Raw']
        return data3
    return dfRet

ma_training=featMA(training_data,bRel=False).fillna(0.0)
#Filling not applicable values with mean instead of NA
#ma_training=rsi_training.fillna(ma_training['ASX.AX'].mean())

In [4]:
#This code is copied from https://github.com/tuckerbalch/QSTK/blob/master/qstkfeat/features.py
#This library does is not supported anymore so I do my manipulation
#This code calculates momentum

def returnize0(nds):
    """
    @summary Computes stepwise (usually daily) returns relative to 0, where
    0 implies no change in value.
    @return the array is revised in place
    """
    s= np.shape(nds)
    if len(s)==1:
        nds=np.expand_dims(nds,1)
    #print(s)
    nds[1:, :] = (nds[1:, :] - nds[0:-1]) / abs(nds[0:-1])
    nds[0, :] = np.zeros(nds.shape[1])
    
    #return nds

def featMomentum(dData, lLookback=20, b_human=False ):
    '''
    @summary: N day cumulative return (based on 1) indicator
    @param dData: Dictionary of data to use
    @param lLookback: Number of days to look in the past
    @param b_human: if true return dataframe to plot
    @return: DataFrame array containing values
    '''
    if b_human:
        for sym in dData['close']:
            x=1000/dData['close'][sym][0]
            dData['close'][sym]=dData['close'][sym]*x
        return dData['close']
    dfPrice = dData['close'].copy()
    #print(dfPrice.values)
    
    #Calculate Returns
    returnize0(dfPrice.values)
    
    #Calculate rolling sum
    dfRet = dfPrice.rolling(lLookback).sum()
    #print(dfRet.head(100))
    
    
    return dfRet

mom_training=featMomentum(training_data).fillna(0.0)
#Filling not applicable values with mean instead of NA
#mom_training=rsi_training.fillna(mom_training['ASX.AX'].mean())
#print(mom_training.head(100))

In [5]:
# This code calculates RSI

def featRSI( dData, lLookback=14,  b_human=False):
    '''
    @summary: Calculate RSI
    @param dData: Dictionary of data to use
    @param lLookback: Number of days to look in the past, 14 is standard
    @param b_human: if true return dataframe to plot
    @return: DataFrame array containing values
    '''

    # create deltas per day
    delta = dData['close'].diff()
    #-----------
    dUp, dDown = delta.copy(), delta.copy()
    dUp[dUp < 0] = 0
    dDown[dDown > 0] = 0

    RolUp = dUp.rolling(14).mean()
    RolDown = dDown.rolling(14).mean().abs()
    

    RS = RolUp / RolDown
    rsi= 100.0 - (100.0 / (1.0 + RS))
    return rsi

rsi_training=featRSI(training_data).fillna(0.0)
#Filling not applicable values with mean instead of NA
#rsi_training=rsi_training.fillna(rsi_training['ASX.AX'].mean())
#print(rsi_training.head(100))

In [6]:
#This code is copied from https://github.com/tuckerbalch/QSTK/blob/master/qstkfeat/features.py
#This library does is not supported anymore so I do my manipulation
#This code calculates exponential moving average

def featEMA( dData, lLookback=20, bRel=True,  b_human=False ):
    '''
    @summary: Calculate exponential moving average
    @param dData: Dictionary of data to use
    @param lLookback: Number of days to look in the past
    @param b_human: if true return dataframe to plot
    @return: DataFrame array containing values
    '''
    
    dfPrice = dData['close']
    
    dfRet = dfPrice.ewm(span=lLookback).mean()
    
    if bRel:
        dfRet = dfRet / dfPrice;
    if b_human:  
        data2 = dfRet*dData['close']
        data3 = pand.DataFrame({"Raw":data2[data2.columns[0]]})
        for sym in dfRet.columns:
            if sym != '$SPX' and sym != '$VIX':
                data3[sym + " Moving Average"] = data2[sym]
                data3[sym] = dData['close'][sym]
        del data3['Raw']
        return data3          
    return dfRet

ema_training=featEMA(training_data,bRel=False).fillna(0.0)
#Filling not applicable values with mean instead of NA
#ema_training=ema_training.fillna(mom_training['ASX.AX'].mean())

In [7]:
#This code is copied from https://github.com/tuckerbalch/QSTK/blob/master/qstkfeat/features.py
#This library does is not supported anymore so I do my manipulation
#This code calculates drawDown

def featDrawDown( dData, lLookback=30,  b_human=False):
    '''
    @summary: Calculate Drawdown for the stock
    @param dData: Dictionary of data to use
    @param lLookback: Days to look back
    @return: DataFrame array containing values
    @param b_human: if true return dataframe to plot
    @warning: Drawdown and RunUp can depend heavily on sample period
    '''
    
    dfPrice = dData['close']
    
    #''' Feature DataFrame will be 1:1, we can use the price as a template '''
    dfRet = pd.DataFrame( index=dfPrice.index, columns=dfPrice.columns, data=np.zeros(dfPrice.shape) )
    
    dfMax = dfPrice.rolling(window=lLookback).max()
    
    return (dfMax - dfPrice) / dfMax;
    
    if b_human:
        for sym in dData['close']:
            x=1000/dData['close'][sym][0]
            dData['close'][sym]=dData['close'][sym]*x
        return dData['close']
    return dfRet

drawDown_training=featDrawDown(training_data).fillna(0.0)
#Filling not applicable values with mean instead of NA
#drawDown_training=drawDown_training.fillna(mom_training['ASX.AX'].mean())
#print(draw_training)

In [8]:
#This code is copied from https://github.com/tuckerbalch/QSTK/blob/master/qstkfeat/features.py
#This library does is not supported anymore so I do my manipulation
#This code calculates drawDown

def featRunUp( dData, lLookback=30, b_human=False ):
    '''
    @summary: CalculateRunup for the stock
    @param dData: Dictionary of data to use
    @param lLookback: Number of days to calculate min over 
    @return: DataFrame array containing feature values
    @param b_human: if true return dataframe to plot
    @warning: Drawdown and RunUp can depend heavily on when the sample starts
    '''
    
    dfPrice = dData['close']
    
    dfMax = dfPrice.rolling(window=lLookback).min()
    return dfPrice / dfMax;
            
    if b_human:
        for sym in dData['close']:
            x=1000/dData['close'][sym][0]
            dData['close'][sym]=dData['close'][sym]*x
        return dData['close']
    return dfRet

runUp_training=featRunUp(training_data).fillna(0.0)
#Filling not applicable values with mean instead of NA
#runUp_training=runUp_training.fillna(mom_training['ASX.AX'].mean())


In [9]:
#This code is copied from https://github.com/tuckerbalch/QSTK/blob/master/qstkfeat/features.py
#This library does is not supported anymore so I do my manipulation
#This code calculates moving average
def featVolumeDelta( dData, lLookback=30, b_human=False ):
    '''
    @summary: Calculate moving average
    @param dData: Dictionary of data to use
    @param lLookback: Number of days to use for MA
    @param b_human: if true return dataframe to plot
    @return: DataFrame array containing values
    '''
    
    dfVolume = dData['volume']
    
    dfRet = dfVolume.rolling(lLookback).mean()
    dfRet /= dfVolume
        
    if b_human:
        for sym in dData['close']:
            x=1000/dData['close'][sym][0]
            dData['close'][sym]=dData['close'][sym]*x
        return dData['close']    
    return dfRet

volume_training=featVolumeDelta(training_data).fillna(0.0)
print(volume_training)
#Filling not applicable values with mean instead of NA
#volume_training=volume_training.fillna(mom_training['ASX.AX'].mean())

              ASX.AX
2014-01-02  0.000000
2014-01-03  0.000000
2014-01-06  0.000000
2014-01-07  0.000000
2014-01-08  0.000000
...              ...
2018-12-21  0.383853
2018-12-24  2.942769
2018-12-27  1.414270
2018-12-28  1.734140
2018-12-31  3.305129

[1266 rows x 1 columns]


In [10]:
#This code is copied from https://github.com/tuckerbalch/QSTK/blob/master/qstkfeat/features.py
#This library does is not supported anymore so I do my manipulation
#This code calculates moving average
def featRand( dData, b_human=False ):
    '''
    @summary: Random feature - used for robustness testing
    @param dData: Dictionary of data to use
    @param b_human: if true return dataframe to plot
    @return: DataFrame array containing values
    '''
    
    dfPrice = dData['close']
    
    #''' Feature DataFrame will be 1:1, we can use the price as a template '''
    dfRet = pd.DataFrame( index=dfPrice.index, columns=dfPrice.columns, 
                            data=np.random.randn(*dfPrice.shape) )
    
    if b_human:
        for sym in dData['close']:
            x=1000/dData['close'][sym][0]
            dData['close'][sym]=dData['close'][sym]*x
        return dData['close']
    return dfRet

random_training=featRand(training_data).fillna(0.0)
#random_training=random_training.fillna(mom_training['ASX.AX'].mean())
#print(random_training)

In [11]:
#This code is copied from https://github.com/tuckerbalch/QSTK/blob/master/qstkfeat/features.py
#This library does is not supported anymore so I do my manipulation
#This code calculates moving average
def featBollinger( dData, lLookback=20, b_human=False ):
    '''
    @summary: Calculate bollinger position as a function of std deviations.
    @param dData: Dictionary of data to use
    @param lLookback: Number of days to calculate moving average over
    @param b_human: if true return dataframe to plot
    @return: DataFrame array containing feature values
    '''
    if b_human:
        dfPrice = dData['close']
        nstdsRet = pd.DataFrame( index=dfPrice.index, columns=dfPrice.columns, data=np.zeros(dfPrice.shape) )
        #average minus standard deviation
        pstdsRet = pd.DataFrame( index=dfPrice.index, columns=dfPrice.columns, data=np.zeros(dfPrice.shape) )      
        data3 = pd.DataFrame({"Raw":dfPrice[dfPrice.columns[0]]})
        for sym in dfPrice.columns:
            if sym != '$SPX' and sym != '$VIX':
                tsPrice = dfPrice[sym]
                nstdRet = nstdsRet[sym]
                pstdRet = pstdsRet[sym]
                for i in range(len(tsPrice.index)):
                    if i < lLookback - 1:
                        nstdRet[i] = float('nan')
                        pstdRet[i] = float('nan')
                        continue    
                    fAvg = np.average( tsPrice[ i-(lLookback-1):i+1 ] )
                    fStd = np.std( tsPrice[ i-(lLookback-1):i+1 ] )
                    pstdRet[i] = fAvg+2.0*fStd
                    nstdRet[i] = fAvg-2.0*fStd  
                data3[sym] = dfPrice[sym]
                data3[sym + " Lower"] = nstdsRet[sym]
                data3[sym + " Upper"] = pstdsRet[sym]
        del data3['Raw']
        return data3
    else:
        dfPrice = dData['close']
        #''' Feature DataFrame will be 1:1, we can use the price as a template '''
        dfRet = pd.DataFrame( index=dfPrice.index, columns=dfPrice.columns, data=np.zeros(dfPrice.shape) )
        
        #''' Loop through stocks '''
        dfAvg = dfPrice.rolling(lLookback).mean()
        dfStd = dfPrice.rolling(lLookback).std()
        return (dfPrice - dfAvg) / (2.0*dfStd)
    
bollinger_training=featBollinger(training_data).fillna(0.0)
#bollinger_training=bollinger_training.fillna(bollinger_training['ASX.AX'].mean())

In [12]:
#If I want to predict the stock price for the next few days
#The current data set has values from 01/01/2019-today
currentDate = dt.date.today()
#YOu can take -30 as this dataset is relatively not needed however it must have 30 days as simple moving average has a lookback of 30 days
oldDate = currentDate + dt.timedelta(days=-30)
raw_current_data = {}
for s in stocks_clone:
    file_name = download_dir + "/"+ s + '_current'+'.csv'
    #print(file_name)
    if(os.path.isfile(file_name)) :
      
        raw_current_data[s] = pd.read_csv(file_name)
    

current_data = sanitizedataforqstk(raw_current_data)

In [13]:
#DEAP LIBRARY IS TAKEN

toolbox = base.Toolbox()
# basic initialization for deap
def initDeap():
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)
    # stochastic removed as High and Low can't be calculated for
    IND_SIZE = 7  
    # registerd coeff alias to coefficient function
    toolbox.register("coeffs", coefficient)
    # individual alias registerd to tools.initRepeat
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.coeffs, n=IND_SIZE)
    # population alias registered
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    # evaluate fucntion registered to computeFitness function
    toolbox.register("evaluate", compute_fitness)
    toolbox.register("mate", tools.cxSimulatedBinary, eta=0.3)
    toolbox.register("mutate", tools.mutGaussian, mu=0.0, sigma=1.0, indpb=0.05)
    toolbox.register("select", tools.selTournament, tournsize=5)

# 1. rsi, 2. ma, 3. mom above is the order of coefficients removed pivot
# point and stochastic oscillator as one of the technical indicator, because
# lets say we are running this for the next 6 months,in that case we would
# not have the value of high, low of future. as such for future we can't use
# it.
def compute_fitness(individual):
    error = 0.0
    count = 0
    for key in stocks:
        # difference in RSI between day1 and day1 + 1
        rsidiff = rsi_training[key] - rsi_training[key].shift()
        # difference in moving average between day1 and day1 + 1
        madiff = ma_training[key] - ma_training[key].shift()
        # difference in momentum between day1 and day1 + 1
        momdiff = mom_training[key] - mom_training[key].shift()
        voldiff = runUp_training[key] - runUp_training[key].shift()
        drawDownDiff=drawDown_training[key]-drawDown_training[key].shift()
        bollingerDiff=bollinger_training[key]- bollinger_training[key].shift()
        emaDiff=ema_training[key]- ema_training[key].shift()
        per_stock_count = 0
        dataframe_index = training_data[qstk_keys[3]][key].index.tolist()
        for (date, val) in rsi_training[key].iteritems():
            next_day = dataframe_index[per_stock_count + 1] if\
                per_stock_count <= len(dataframe_index) - 1 else None
            if not np.isnan(rsidiff[date]) and next_day is not None:
                #            Fitness defined as
                # market close (day2) - [ rsi difference between consecutive day * (coeff 1) +
                # moving average difference between consecutive day * (coeff 2) +
                # momentum difference between consecutive day * (coeff 3) +
                
                # Objective is to minimize the summation of squares of the above differences
                ft = (rsidiff[date]) * individual[0] + (madiff[date]) * individual[1] \
                     + (momdiff[date]) * individual[2]+ (voldiff[date]) * individual[3]+ (drawDownDiff[date]) * individual[4] \
                    + (bollingerDiff[date]) * individual[5]+ (emaDiff[date]) * individual[6]
                ft = ft + training_data[qstk_keys[3]][key][date]
                error = error + np.square(training_data[qstk_keys[3]][key][next_day] - ft)
                count = count + 1
                per_stock_count = per_stock_count + 1
    # dividing by total count gives the mean square error, which we want to minimize
    #print("error")
    #print(error/count)
    return (error / count,)


def coefficient():
    return random.random()

In [14]:

#Get our base model up and Running
#Selecting the best set of features amongst the 9 features
#Start with a base model of two
#features available-
#bollinger_training,random_training,volume_training,runUp_training,drawDown_training,mom_training
#rsi_training,ma_training,ema_training


if __name__ == '__main__':
    
    bestFitness=10.0

    base_features=[ma_training,mom_training,rsi_training]
    base_featureString=["ma_training","mom_training","rsi_training"]

    features_bucket_left=[random_training,volume_training,runUp_training,drawDown_training,bollinger_training,
    ema_training]
    
    
    
    #=============#
    
    #random_training increased mean square error
    
    
    initDeap()
    # population generated
    #========================#
    #First pass we run our base Model
    pop = toolbox.population(n=50)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    stats.register("max", np.max)
    stats.register("std", np.std)
    stats.register("median", np.median)
    # built in algo used with cross over probability = 0.5, mutation probability = 0.2 and number of generation = 5
    pop, logbook = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=5, stats=stats, halloffame=hof,
                                       verbose=True)
    features = ','.join(base_featureString)
    print("Best individual with feature-set: %s \nwith fitness: %s" % (features, hof[0].fitness))
    
    #maxErr=
    

gen	nevals	avg   	min     	max    	std    	median 
0  	50    	17.559	0.508895	45.9606	13.7674	13.7783
1  	30    	4.67422	0.508895	18.5917	4.8628 	3.60859
2  	36    	1.39964	0.20234 	12.8263	1.99421	0.898742
3  	31    	0.47194	0.0668253	0.901938	0.158186	0.508895
4  	34    	7.17947	0.0291335	131.886 	27.3509 	0.408584
5  	26    	0.170511	0.0291335	1.13214 	0.203058	0.106453
Best individual with feature-set: ma_training,mom_training,rsi_training 
with fitness: (0.029133461116427947,)


<table style="width:100%">
  <tr>
    <th>Features</th>
    <th>Mean Square Error</th> 
    
  </tr>
  <tr>
    <td>Simple Moving Average, RSI,Momentum</td>
    <td>0.0982</td> 
    
  </tr>
  <tr>
    <td>Simple Moving Average, RSI,Momentum,Random Indicator</td>
    <td>0.6101</td>
  </tr>
   <tr>
    <td>Simple Moving Average, RSI,Momentum,Run Up</td>
    <td>0.0309</td>
  </tr>
   <tr>
    <td>Simple Moving Average, RSI,Momentum,Run Up,Draw Down</td>
    <td>0.0303</td>
  </tr>
  <tr>
    <td>Simple Moving Average, RSI,Momentum,Run Up,Bollinger Bands</td>
    <td>0.0281</td>
  </tr>
  <tr>
    <td>Simple Moving Average, RSI,Momentum,Run Up,Bollinger Bands,Exponential Moving Average</td>
    <td>0.0291</td>
  </tr>
</table>

In [15]:
<h3 style="red"> From the above table the features; Simple Moving Average, RSI,Momentum,Run Up and the Bollinger Bands gave the lowest mean square error and would be taken for further investigation </h3>

SyntaxError: invalid syntax (<ipython-input-15-b3b83d53e27e>, line 1)